In [8]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import *

In [9]:
# read glossary
glossary = pd.read_json('../data/metadata2kg/round2/r2_glossary.jsonl', lines=True)

# read glossary
glossary = pd.read_json('../data/metadata2kg/round2/r2_glossary.jsonl', lines=True)
glossary['id_no_prefix'] = glossary['label'].str.replace(' ', '')
glossary['id_no_prefix']  = glossary['id_no_prefix'].str.replace('\'', '')
# if id_no_prefix has duplicates, add a number to the end
glossary['c'] = glossary.groupby('id_no_prefix').cumcount()
glossary['c'] = glossary['c'].apply(lambda x: '' if x == 0 else str(x))
glossary['id_no_prefix'] = glossary['id_no_prefix'] + glossary['c']
glossary = glossary.drop(columns=['c'])

In [10]:
glossary[glossary['label'] == 'Ignore']

,id,label,desc,id_no_prefix
906,906,Ignore,"meaningless, ignore it",Ignore


In [11]:
# change id 906
glossary.loc[glossary['label'] == 'Ignore', 'desc'] = 'something that is meaningless or can be ignored'
glossary.loc[glossary['label'] == 'Ignore', 'id_no_prefix'] = 'Ignore0'

In [12]:
glossary[glossary['label'] == 'Ignore']

,id,label,desc,id_no_prefix
906,906,Ignore,something that is meaningless or can be ignored,Ignore0


In [6]:
# save
glossary.to_csv('../data/metadata2kg/round2/r2_glossary_processed.csv', index=False)

## Metadata enrichment

In [13]:
# read metadata
metadata = pd.read_json('../data/metadata2kg/round1/r1_test_metadata.jsonl', lines=True)

In [14]:
metadata_prompt = """Can you clean up the following? Fix mistakes and put everything in english if not. Write abbreviations or acronyms in full and add the abbreviation in brackets after the word. Convert names to their type. Add a relation e.g. is a, of a or other. You must understand the meaning of the word first to write the relation.

### Example
Q: 
0. year, film
1. developer, videogame
2. harelbeke, country

A: 
0. Year of a film
1. Developer of a videogame
3. City (Harelbeke) of a country

### Data

{data}
"""

In [15]:
metadata_desc = {}

In [ ]:
import asyncio

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)
    return wrapped

@background
def fn(groups):
    for chunk in groups:
        if len(set(chunk['id']).intersection(metadata_desc.keys())) > 0:
            print('Already processed')
            return
        
        messages = [
            {"role": "user", "content": metadata_prompt.format(data=print_descriptions((chunk['label'] + ', ' + chunk['table_name']).tolist()))},
        ]

        print(messages[0]['content'])
        m = message_gpt(messages, temperature=0.0, seed=47)
        content = extract_content(m)

        if len(content) != len(chunk):
            print('Error: Length mismatch')
            return
        
        for id, content in zip(chunk['id'], content):
            metadata_desc[id] = content

n_clients = 20

# group metadata in chunks of 100
grouped = metadata.groupby([i//100 for i in range(len(metadata))])

for i in range(n_clients):
    group_idx = np.arange(len(grouped))%n_clients==i
    fn([grouped.get_group(i) for i in range(len(grouped)) if group_idx[i]])

In [ ]:
# save metadata descriptions as json
with open('metadata_descriptions_cleaned.json', 'w') as f:
    json.dump(metadata_desc, f)